# Paper Implementation
Description: This is implementation of paper from S.Saadatnejad et al. "LSTM-Based ECG Classification for Continuous Monitoring on Personal Wearable Devices" [(source)](https://ieeexplore.ieee.org/abstract/document/8691755)
  * They also have source code and preprocessed dataset for dual lead: http://lis.ee.sharif.edu/pub/2020_jbhi_soh/

### TODO next
- [x] implementation for single lead using MITBIH dataset (selfmade implementation preprocessing method based on author paper)
- [ ] Add additional explanation and information

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";


In [8]:
import csv
from glob import glob
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import gc
import pickle
import pathlib
import io
from math import sqrt


from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,Callback
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import concatenate,Input
from tensorflow.keras import optimizers, losses, activations
from tensorflow.keras.models import Model

from tensorflow.keras import backend as K

print(tf.__version__)

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.neural_network import MLPClassifier as mlp
import joblib as jl #import dump, load
from datetime import datetime

#from calcCM import *
import calcCM as cl

1.14.0


In [3]:
## limit GPU memory usage
configproto = tf.compat.v1.ConfigProto()
configproto.gpu_options.per_process_gpu_memory_fraction = 0.3
configproto.gpu_options.allow_growth = True

#limit CPU logical core usage
configproto.intra_op_parallelism_threads=2
configproto.inter_op_parallelism_threads=2

#setup session config
sess = tf.compat.v1.Session(config=configproto)
tf.compat.v1.keras.backend.set_session(sess)

In [4]:
dirPath = os.path.abspath(os.getcwd())
print(dirPath)
#print(pathlib.Path().absolute())

C:\Users\Arta\py\forgit\saad\CODE


In [9]:
now = datetime.now()
dt_string_daytime = now.strftime("%Y%m%d_%H%M%S")
dt_string_daytime +='_saad_selfmade'
print("date and time =", dt_string_daytime)

date and time = 20200908_203123_saad_selfmade


In [10]:
data_result_path = list()
data_result_path.append('model_'+dt_string_daytime)
data_result_path.append(data_result_path[0]+'/ckpt')
data_result_path.append(data_result_path[0]+'/fitHistory')
data_result_path.append(data_result_path[0]+'/modelSummary')

for i in range(0,len(data_result_path)):
    if not os.path.exists(data_result_path[i]):
        print("create new folder "+data_result_path[i])
        os.makedirs(data_result_path[i])
    else:
        print(data_result_path[i]+" already created")

create new folder model_20200908_203123_saad_selfmade
create new folder model_20200908_203123_saad_selfmade/ckpt
create new folder model_20200908_203123_saad_selfmade/fitHistory
create new folder model_20200908_203123_saad_selfmade/modelSummary


In [11]:
currentPath = dirPath+'/'+data_result_path[0]
print(currentPath)

C:\Users\Arta\py\forgit\saad\CODE/model_20200908_203123_saad_selfmade


In [12]:
def activation_func(pos):
    if(pos== 1):
        return activations.relu
    elif(pos==2):
        return activations.sigmoid
    elif(pos==3):
        return activations.softmax
    elif(pos==4):
        return activations.tanh
    else:
        return activations.relu
    
def opt_func(pos,lr=0.0001):
    if(pos==1):
        return optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)
    elif(pos==2):
        return optimizers.SGD(lr=lr, momentum=0.9, decay=0.0, nesterov=False)
    elif(pos==3):
        return optimizers.Nadam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    elif(pos==4):
        return optimizers.Adamax(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    else:
        return optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [13]:
#LSTM hyperparam
modelID = 0
batchList = [100]
epochList = [100]
neuronListA = [30];
neuronListB = [50];
activList = [4]
denseActivList = [3]
optfList=[1]
lr = 1e-3
nstepsA = 10
nstepsB = 10
nstepsC = 5
nclass = 7

In [14]:
#setup LSTM model hyperparam
neuronA = neuronListA[0]
neuronB = neuronListB[0]
actf = activList[0]
dense_actf = denseActivList[0]
optf = optfList[0]

In [15]:
global inp1,inp2,inp3,inp4

def createModelAlpha(timesteps, nInput):
    inp1 = Input(shape=(timesteps[0], nInput[0]),name='inp1')
    inp2 = Input(shape=(timesteps[1], nInput[1]),name='inp2')
    
    
    #model a1
    #modela1 = LSTM(neuronA, activation=activation_func(actf), name='a1_1')(inp1)
    modela1 = LSTM(neuronA, name='a1_1')(inp1)
    #modela1 = LSTM(neuronA,activation_func(actf),name='a1_2')(modela1)

    #modela2
    #modela2 = LSTM(neuronA, activation=activation_func(actf),name='a2_1')(inp2)
    modela2 = LSTM(neuronA, name='a2_1')(inp2)
    #modela2 = LSTM(neuronA,activation_func(actf),name='a2_2')(modela2)
    #modela2 = Flatten()(modela2)

    #model a Merge and dense
    mergeA = concatenate([modela1,modela2],name='merge_a1a2')
    modelAout = Dense(nclass,activation=activation_func(dense_actf), name='modelA_dense')(mergeA)

    model = Model(inputs=[inp1, inp2], outputs=modelAout)

    model.summary()
    model.compile(optimizer=opt_func(optf,lr), 
                  loss='categorical_crossentropy',
                  metrics=["acc"])
    return model

def createModelBeta(timesteps, nInput):

    ##=============
    inp3 = Input(shape=(timesteps[2], nInput[2]),name='inp3')
    
    #single layer LSTM
    ##=============
    #model b and dense
    #modelb = LSTM(neuronB, activation=activation_func(actf), name='b_1')(inp3)
    modelb = LSTM(neuronB, name='b_1')(inp3)
    #modelb = LSTM(neuronB,activation_func(actf),name='b_2')(modelb)
    modelBout = Dense(nclass,activation=activation_func(dense_actf), name='modelB_dense')(modelb)

    model = Model(inputs=[inp3], outputs=modelBout)

    model.summary()
    model.compile(optimizer=opt_func(optf,lr), 
                  loss='categorical_crossentropy',
                  metrics=["acc"])
    return model

def createModelBlend():
    inp4 = Input(shape=(14),name='inp4')
    
    ##=============
    #merge ab result
    mergeAB = Dense(80,activation=activation_func(dense_actf), name='hidden1')(inp4)
    mergeAB = Dense(10,activation=activation_func(dense_actf), name='hidden2')(mergeAB)
    modelABout = Dense(nclass,activation=activation_func(dense_actf), name='modelAB_dense')(mergeAB)

    model = Model(inputs=[inp4], outputs=modelABout)

    model.summary()
    model.compile(optimizer=opt_func(optf,lr), 
                  loss='categorical_crossentropy',
                  metrics=["acc"])
    return model

In [16]:
def dataPadMultiple(sigs,mod):
    numpad = mod - (sigs.shape[1]%mod)
    
    pad = np.zeros((sigs.shape[0],sigs.shape[1]+numpad), dtype=np.float32)
    pad[:sigs.shape[0],:sigs.shape[1]] = sigs
    
    padded_beat = pad
    
    return padded_beat

In [17]:
allData = ['100', '101', '103', '105', '106', '107', '108', '109', '111', 
           '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124',
           '200', '201', '202', '203', '205', '207', '208', '209', '210',
           '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']

excludeList = [102,104,107,217]

patientIdList = allData

rootDataFolder = ""

In [ ]:
def runModified(model_type,model=0,modelAA=0,modelBB=0):
    
    for idx_ptnId,ptnId in enumerate(patientIdList):

        if(int(ptnId) in excludeList):
            print(ptnId+' is excluded ###################################')
            continue;
        print('Load Data -- '+str(ptnId)+'-----------')

        # ======================== load data
        
        # original
        trainXa = np.load(rootDataFolder+'/trainingData/'+ptnId+'_A1_saad_MLII_X.npy') #a1
        trainXb = np.load(rootDataFolder+'/trainingData/'+ptnId+'_A2_saad_MLII_X.npy') #a2
        trainXc = np.load(rootDataFolder+'/trainingData/'+ptnId+'_B_saad_PCA_MLII_X.npy') #b
        trainY = np.load(rootDataFolder+'/trainingData/'+ptnId+'_MLII_Y.npy')

        testXa = np.load(rootDataFolder+'/testingData/'+ptnId+'_A1_saad_MLII_X.npy') #a1
        testXb = np.load(rootDataFolder+'/testingData/'+ptnId+'_A2_saad_MLII_X.npy') #a2
        testXc = np.load(rootDataFolder+'/testingData/'+ptnId+'_B_saad_PCA_MLII_X.npy') #b
        testY = np.load(rootDataFolder+'/testingData/'+ptnId+'_MLII_Y.npy')
        
        trXa = trainXa
        trXb = trainXb
        trXc = trainXc
        trY = trainY

    # ## checking purpose
    #     print(trXa1.shape)
    #     print(trXa2.shape)
    #     print(trXb.shape)
    #     print(trY.shape)

    #     print('========')
    #     print(testXa1.shape)
    #     print(testXa2.shape)
    #     print(testXb.shape)
    #     print(testY.shape)
    
        unique, counts = np.unique(trainY, return_counts=True)
        print(np.asarray((unique, counts)).T)
        print(np.sum(counts))
        print()

        timesteps_a = nstepsA
        timesteps_b = nstepsB
        timesteps_c = nstepsC
        
        trXa = dataPadMultiple(trXa,timesteps_a)
        trXb = dataPadMultiple(trXb,timesteps_b)
        trXc = dataPadMultiple(trXc,timesteps_c)
        
        testXa = dataPadMultiple(testXa,timesteps_a)
        testXb = dataPadMultiple(testXb,timesteps_b)
        testXc = dataPadMultiple(testXc,timesteps_c)

        #===================== test data preparation

        nInput_a = testXa.shape[1]//timesteps_a
        nInput_b = testXb.shape[1]//timesteps_b
        nInput_c = testXc.shape[1]//timesteps_c

        
        #test data model.evaluation
        testXa = testXa.reshape(testXa.shape[0],timesteps_a,nInput_a)
        testXb = testXb.reshape(testXb.shape[0],timesteps_b,nInput_b)
        testXc = testXc.reshape(testXc.shape[0],timesteps_c,nInput_c)
        testY = to_categorical(testY.astype(np.int),nclass)
        


        #================== train data preparation

        nInput_a = trXa.shape[1]//timesteps_a
        nInput_b = trXb.shape[1]//timesteps_b
        nInput_c = trXc.shape[1]//timesteps_c

        trXa = trXa.reshape(trXa.shape[0],timesteps_a,nInput_a)
        trXb = trXb.reshape(trXb.shape[0],timesteps_b,nInput_b)
        trXc = trXc.reshape(trXc.shape[0],timesteps_c,nInput_c)
        trY = to_categorical(trY.astype(np.int),nclass)
        
        #define input model
        timestepsList = list()
        timestepsList.append(timesteps_a)
        timestepsList.append(timesteps_b)
        timestepsList.append(timesteps_c)
        nInputList = list()
        nInputList.append(nInput_a)
        nInputList.append(nInput_b)
        nInputList.append(nInput_c)

        print(trY.shape)

        print('######### Training for patient '+str(ptnId)+'--############')

        # training duration
        batch_size = batchList[0]
        num_batches_per_epoch = int(np.ceil(trainY.shape[0]/batch_size))
        epoch = epochList[0]
        
        clear_session = True
        if clear_session:
            tf.keras.backend.clear_session()
        model_code = ['_aa','_bb']
        # code_idx => for model checkpoint naming purpose for model alpha and beta
        # model_type => for check point naming purpose for model blend
        if model == 0:
            if(model_type == 1):
                code_idx = 0 
                model = createModelAlpha(timestepsList,nInputList)
            elif(model_type == 2):
                code_idx = 1
                model = createModelBeta(timestepsList,nInputList)
            elif(model_type == 3 or model_type == 4 or model_type == 5):
                code_idx = 2
                model = createModelBlend()
            
        
        file_path = currentPath+"/ckpt/"+str(ptnId)
    
        if(code_idx <= 1):
            flag_model = model_code[code_idx]
            ckpttype = ['min_loss'+flag_model,'max_acc'+flag_model,'last'+flag_model]
            fpl = list() #file path list
            for i in range(0, len(ckpttype)):
                fpl.append(file_path+"_"+ckpttype[i]+".h5")


            checkpoint_timed = list()
            checkpoint_last = ModelCheckpoint(fpl[2], monitor='acc',period=epoch, verbose=1, save_best_only=False, mode='max')
            checkpoint_min_loss = ModelCheckpoint(fpl[0], monitor='loss', verbose=1, save_best_only=True, mode='min')
            checkpoint_max_acc = ModelCheckpoint(fpl[1], monitor='acc', verbose=1, save_best_only=True, mode='max')

            callbacks_list = [checkpoint_min_loss,checkpoint_max_acc,checkpoint_last]
          
        Wsave = model.get_weights()
        
        if(model_type == 1):
            ## AA
            history = model.fit([trXa,trXb], trY,batch_size=num_batches_per_epoch, epochs=epoch,
                                callbacks=callbacks_list, verbose=1,shuffle=True,
                                workers=1, use_multiprocessing=False)
        elif(model_type == 2):
            ##BB
            history = model.fit([trXc], trY,batch_size=num_batches_per_epoch, epochs=epoch,
                                callbacks=callbacks_list, verbose=1,shuffle=True,
                                workers=1, use_multiprocessing=False)
        elif(model_type == 3 or model_type == 4 or model_type == 5):
            ##blend
            # predict each model aplha and mdoel beta then combine the result prediction
            model_root_dir = data_result_path[0] # untuk direct root folder
            ckpt_root_dir = model_root_dir+'/ckpt'
            if modelAA == 0:
                modelAA = createModelAlpha(timestepsList,nInputList)
            if modelBB == 0:
                modelBB = createModelBeta(timestepsList,nInputList)
                
            if model_type == 3:
                modelAA.load_weights(ckpt_root_dir+'/'+str(ptnId)+'_max_acc_aa.h5')
                modelBB.load_weights(ckpt_root_dir+'/'+str(ptnId)+'_max_acc_bb.h5')
                ckpt_path = file_path+"_max_acc_last_blend.h5"
            elif model_type == 4:
                modelAA.load_weights(ckpt_root_dir+'/'+str(ptnId)+'_min_loss_aa.h5')
                modelBB.load_weights(ckpt_root_dir+'/'+str(ptnId)+'_min_loss_bb.h5')
                ckpt_path = file_path+"_min_loss_last_blend.h5"
            elif model_type == 5:
                modelAA.load_weights(ckpt_root_dir+'/'+str(ptnId)+'_last_aa.h5')
                modelBB.load_weights(ckpt_root_dir+'/'+str(ptnId)+'_last_bb.h5')
                ckpt_path = file_path+"_last_last_blend.h5"
                
            print(ckpt_path)
            
            predResultAA = modelAA.predict_on_batch([trXa,trXb])
            predResultBB = modelBB.predict_on_batch([trXc])
            
            if(np.size(predResultAA,0)!=np.size(predResultBB,0)):
                predResultBB = predResultBB[:predResultAA.shape[0],:]
#             print(testXa.shape)
#             print(testXc.shape)

            mergeAABB = np.concatenate((predResultAA,predResultBB),axis=1)
            ep = 200 # default 200
        
            # mlp sklearn ==========================
            clf = 0
            clf = mlp(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(80,10), random_state = 1)
            clf.fit(mergeAABB, trY)
            jl.dump(clf, ckpt_path[:-3]+".jblib")
            

        model.set_weights(Wsave)
    return 0

In [ ]:
## personal note
pnote = 'saadajad single lead implmenetation -> code selfmade, data selfmade\\n'
pathNote = currentPath+"\PNOTE.txt"
with open(pathNote,'a') as f:
        f.write(pnote)

In [ ]:
# run training
for i in range(1,6):
    runModified(i)

In [ ]:
# model checkpoint directory

#model_root_dir = 'model_20200402_181004_ori_sm_data_ori_selfmade'
model_root_dir = data_result_path[0] # untuk direct root folder
ckpt_root_dir = model_root_dir+'/ckpt'
nclass = 7

In [ ]:
model = 0
print("###### "+ckpt_root_dir)
custom_msg  = model_root_dir[5:]
ckpttype = ['last','min_loss','max_acc']
#ckpttype = ['min_loss','max_acc']
str_result = ""
modelAA = 0
modelBB = 0
for i,ckpt_val in enumerate(ckpttype):
    cm = list()
    for idx_ptnId,ptnId in enumerate(patientIdList):
        if(int(ptnId) in excludeList):
            print(ptnId+' is excluded ###################################')
            continue;
        
        testXa = np.load(rootDataFolder+'/testingData/'+ptnId+'_A1_saad_MLII_X.npy') #a1
        testXb = np.load(rootDataFolder+'/testingData/'+ptnId+'_A2_saad_MLII_X.npy') #a2
        testXc = np.load(rootDataFolder+'/testingData/'+ptnId+'_B_saad_PCA_MLII_X.npy') #b
        testY = np.load(rootDataFolder+'/testingData/'+ptnId+'_MLII_Y.npy')

        timesteps_a = nstepsA
        timesteps_b = nstepsB
        timesteps_c = nstepsC
        
        testXa = dataPadMultiple(testXa,timesteps_a)
        testXb = dataPadMultiple(testXb,timesteps_b)
        testXc = dataPadMultiple(testXc,timesteps_c)

        #===================== test data preparation

        nInput_a = testXa.shape[1]//timesteps_a
        nInput_b = testXb.shape[1]//timesteps_b
        nInput_c = testXc.shape[1]//timesteps_c

        
        #test data model.evaluation
        testXa = testXa.reshape(testXa.shape[0],timesteps_a,nInput_a)
        testXb = testXb.reshape(testXb.shape[0],timesteps_b,nInput_b)
        testXc = testXc.reshape(testXc.shape[0],timesteps_c,nInput_c)
        testY = to_categorical(testY.astype(np.int),nclass)
    
        
        
        timestepsList = list()
        timestepsList.append(timesteps_a)
        timestepsList.append(timesteps_b)
        timestepsList.append(timesteps_c)
        nInputList = list()
        nInputList.append(nInput_a)
        nInputList.append(nInput_b)
        nInputList.append(nInput_c)


        if (model == 0):
            model = createModelBlend()
        
        if modelAA == 0:
            modelAA = createModelAlpha(timestepsList,nInputList)
        if modelBB == 0:
            modelBB = createModelBeta(timestepsList,nInputList)

        modelAA.load_weights(ckpt_root_dir+'/'+str(ptnId)+'_'+ckpt_val+'_aa.h5')
        modelBB.load_weights(ckpt_root_dir+'/'+str(ptnId)+'_'+ckpt_val+'_bb.h5')

        predResultAA = modelAA.predict([testXa,testXb])
        predResultBB = modelBB.predict([testXc])

        if(np.size(predResultAA,0)!=np.size(predResultBB,0)):
            predResultBB = predResultBB[:predResultAA.shape[0],:]

        
        mergeAABB = np.concatenate((predResultAA,predResultBB),axis=1)
        
        #sklearn mlp predict =================
        clf = 0
        clf = jl.load(ckpt_root_dir+'/'+str(ptnId)+'_'+ckpt_val+'_last_blend.jblib')
        predResult = clf.predict(mergeAABB)
        
        
        print("ptnid {} predshape{}".format(ptnId,predResult.shape))

        ytrue = cl.predictionToLabel(testY,nclass)
        ypred = cl.predictionToLabel(predResult,nclass)
        cm.append( confusion_matrix(ytrue, ypred,np.arange(nclass)) )

    cm_final = np.zeros_like(cm[0])
    for idx,idxval in enumerate(cm):
        cm_final = cm_final+idxval 
    
    print(cm_final)
    print()
    tmp_str_result, sumResult= cl.cm_code_paper(model_root_dir,cm_final,ckpt_val)
    str_result +=tmp_str_result
    
cl.saveResult(model_root_dir,str_result,custom_msg)

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")